# Beilinson-Bloch Heights Demo

This notebook demonstrates the use of Beilinson-Bloch regulatory height pairings for verifying Poitou-Tate compatibility (PT) in ranks r ≥ 2.

## Theoretical Background

For elliptic curves of rank r ≥ 2, the Birch–Swinnerton-Dyer conjecture involves the **regulator**, which is the determinant of the height pairing matrix:

$$\text{Reg} = \det(\langle P_i, P_j \rangle)_{i,j=1}^r$$

where $P_1, \ldots, P_r$ are independent generators of $E(\mathbb{Q})$ modulo torsion.

The Beilinson-Bloch construction provides a canonical way to compute these height pairings via higher Chow cycles.

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

from sage.all import EllipticCurve
from src.beilinson_bloch_heights import (
    BeilinsonBlochHeightPairing,
    BeilinsonBlochVerifier,
    batch_verify_beilinson_bloch,
    find_high_rank_curves
)

import numpy as np
import matplotlib.pyplot as plt

## Example 1: Rank 1 Curve (11a1)

First, let's verify a rank 1 curve to understand the basics.

In [ ]:
# Rank 1 example
E = EllipticCurve('11a1')
print(f"Curve: {E}")
print(f"Conductor: {E.conductor()}")
print(f"Rank: {E.rank()}")

# Get generator
gens = E.gens()
print(f"\nGenerators: {gens}")

# Initialize height pairing
height_pairing = BeilinsonBlochHeightPairing(E)

if gens:
    P = gens[0]
    h_PP = height_pairing.compute_height_pairing(P, P)
    print(f"\nHeight ⟨P,P⟩ = {h_PP:.6f}")
    
    # Compare with canonical height
    h_canonical = P.height()
    print(f"Canonical height ĥ(P) = {h_canonical:.6f}")

## Example 2: Rank 2 Curve (389a1)

Now let's look at a rank 2 curve, which requires computing a 2×2 height matrix.

In [ ]:
# Rank 2 example
E = EllipticCurve('389a1')
print(f"Curve: {E}")
print(f"Conductor: {E.conductor()}")
print(f"Rank: {E.rank()}")

# Get generators
gens = E.gens()
print(f"\nNumber of generators: {len(gens)}")

if len(gens) >= 2:
    # Compute height matrix
    height_pairing = BeilinsonBlochHeightPairing(E)
    H = height_pairing.compute_height_matrix(gens)
    
    print(f"\nHeight matrix:")
    print(H)
    
    # Compute regulator
    reg = height_pairing.compute_regulator(gens)
    print(f"\nRegulator (Beilinson-Bloch): {reg:.6f}")
    
    # Compare with SageMath regulator
    try:
        reg_sage = E.regulator()
        print(f"Regulator (SageMath): {reg_sage:.6f}")
        print(f"Relative difference: {abs(reg - reg_sage)/reg_sage * 100:.2f}%")
    except:
        print("SageMath regulator not available")

## Example 3: Regulator Verification

Use the verifier to check compatibility between algebraic and analytic regulators.

In [ ]:
# Verify regulator compatibility
E = EllipticCurve('389a1')
verifier = BeilinsonBlochVerifier(E)

result = verifier.verify_regulator_compatibility()

print("=== Regulator Verification ===")
print(f"Rank: {result['rank']}")
print(f"Algebraic regulator: {result['algebraic_regulator']:.6f}")
print(f"Analytic regulator: {result['analytic_regulator']:.6f}")
print(f"Compatible: {result['compatible']}")
print(f"Relative error: {result['relative_error']*100:.2f}%")

## Example 4: Generate Certificate

Generate a formal LaTeX certificate for the verification.

In [ ]:
# Generate certificate
certificate = verifier.generate_certificate(result)

# Save certificate
filename = f'certificate_BB_heights_{E.label()}.tex'
with open(filename, 'w') as f:
    f.write(certificate)

print(f"Certificate saved to: {filename}")
print("\n=== Certificate Preview ===")
print(certificate[:500] + "...")

## Example 5: Batch Verification

Verify multiple curves with ranks ≥ 2.

In [ ]:
# Find high-rank curves
print("Finding curves with rank ≥ 2...")
high_rank_curves = find_high_rank_curves(conductor_limit=500, target_rank=2)
print(f"Found {len(high_rank_curves)} curves")
print(f"First 10: {high_rank_curves[:10]}")

# Batch verification on a subset
test_curves = high_rank_curves[:5]  # First 5 curves
print(f"\nVerifying {len(test_curves)} curves...")

batch_results = batch_verify_beilinson_bloch(test_curves, max_rank=3)

print("\n=== Batch Results ===")
print(f"Total curves: {batch_results['total_curves']}")
print(f"Verified: {batch_results['verified']}")
print(f"Success rate: {batch_results['success_rate']*100:.1f}%")

## Example 6: Visualization

Visualize the height matrix for a rank 2 curve.

In [ ]:
# Visualize height matrix
E = EllipticCurve('389a1')
gens = E.gens()

if len(gens) >= 2:
    height_pairing = BeilinsonBlochHeightPairing(E)
    H = height_pairing.compute_height_matrix(gens)
    
    plt.figure(figsize=(8, 6))
    plt.imshow(H, cmap='viridis', aspect='auto')
    plt.colorbar(label='Height value')
    plt.title(f'Height Matrix for {E.label()}')
    plt.xlabel('Generator index')
    plt.ylabel('Generator index')
    
    # Add values as text
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            plt.text(j, i, f'{H[i,j]:.2f}', 
                    ha='center', va='center', color='white', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(f'height_matrix_{E.label()}.png', dpi=150)
    plt.show()
    
    print(f"Height matrix saved as height_matrix_{E.label()}.png")

## Example 7: Detailed Analysis

Analyze the verification results for all tested curves.

In [ ]:
# Detailed analysis
print("=== Detailed Verification Results ===")
print()

for curve_label, result in batch_results['results'].items():
    print(f"Curve: {curve_label}")
    print(f"  Rank: {result['rank']}")
    print(f"  Algebraic reg: {result['algebraic_regulator']:.6f}")
    print(f"  Analytic reg: {result['analytic_regulator']:.6f}")
    print(f"  Compatible: {'✓' if result['compatible'] else '✗'}")
    print()

## Summary

This notebook demonstrated:

1. **Height Pairing Computation**: Computing ⟨P, Q⟩_BB for points on E
2. **Height Matrix**: Building the matrix for rank ≥ 2 curves
3. **Regulator Computation**: det(H) as the Beilinson-Bloch regulator
4. **Verification**: Comparing with analytic regulator from L-functions
5. **Certificate Generation**: Formal LaTeX certificates
6. **Batch Processing**: Verification of multiple high-rank curves

### Next Steps

- Integrate with modular symbols for explicit cycle construction
- Add PARI/GP interface for divisor computations
- Extend to larger conductor ranges (N ≤ 1000)
- Connect with Poitou-Tate duality verification